In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import pathlib
import os

# --- Configuration ---
ZIPPED_LIST = json.loads(pathlib.Path("diced_particle_systems.json").read_text())
ITEM_LIST = [name for name, definition in ZIPPED_LIST]
DEFINITIONS = {name: definition or "None" for name, definition in ZIPPED_LIST}
OUTPUT_FILENAME = "diced_particle_systems_colors.json"
SIZE_OPTIONS = ['Fine (1cm)', 'Coarse (5cm)']

# --- Global variables to keep track of state ---
current_item_index = 0
results_data = {}

# --- Load existing data if the JSON file exists ---
if os.path.exists(OUTPUT_FILENAME):
    try:
        with open(OUTPUT_FILENAME, 'r') as f:
            results_data = json.load(f)
        print(f"Loaded existing data from {OUTPUT_FILENAME}")
    except json.JSONDecodeError:
        print(f"Warning: {OUTPUT_FILENAME} contains invalid JSON. Starting with empty data.")
    except Exception as e:
        print(f"Error loading {OUTPUT_FILENAME}: {e}. Starting with empty data.")


# --- Widgets ---
item_label = widgets.Label()
definition_label = widgets.Label()
color_picker = widgets.ColorPicker(
    concise=False,
    description='Select Color:',
    value='lightblue',
    disabled=False
)
radio_buttons = widgets.RadioButtons(
    options=SIZE_OPTIONS,
    description='Select Option:',
    disabled=False
)
save_button = widgets.Button(
    description="SAVE",
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save current selection and move to next item',
    icon='check'
)
skip_button = widgets.Button(
    description="SKIP",
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Skip to next item',
    icon='arrow-right'
)
output_area = widgets.Output() # To display messages like "Saved!" or "All items processed!"

# --- Event Handler for Save Button ---
def on_button_clicked(should_save):
    global current_item_index
    global results_data

    with output_area:
        clear_output(wait=True) # Clear previous messages in this output area
        if should_save:
            current_item = ITEM_LIST[current_item_index]
            selected_color = color_picker.value
            selected_radio = radio_buttons.value

            # Store the data
            results_data[current_item] = {
                "color": selected_color,
                "option": selected_radio
            }

            # Save to JSON file
            try:
                with open(OUTPUT_FILENAME, 'w') as f:
                    json.dump(results_data, f, indent=4)
                print(f"Saved: {current_item} - Color: {selected_color}, Option: {selected_radio}")
                print(f"Data saved to {OUTPUT_FILENAME}")
            except Exception as e:
                print(f"Error saving data: {e}")
                return # Do not proceed if saving failed

        # Move to the next item
        current_item_index += 1
        if current_item_index < len(ITEM_LIST):
            display_next_item()
        else:
            item_label.value = "All items processed! 🎉"
            definition_label.value = ""
            color_picker.disabled = True
            radio_buttons.disabled = True
            save_button.disabled = True
            skip_button.disabled = True
            print("All items have been annotated.")

# --- Function to display widgets for the current item ---
def display_next_item():
    if current_item_index < len(ITEM_LIST):
        current_item = ITEM_LIST[current_item_index]
        item_label.value = f"Current Item: {current_item.upper()}"
        definition_label.value = f"Definition: {DEFINITIONS[current_item]}"

        # Reset widget states for the new item if needed (e.g., default color/option)
        # Or, try to load previously saved values for this item if it exists in results_data
        if current_item in results_data:
            color_picker.value = results_data[current_item].get("color", 'lightblue')
            # Ensure the radio button value is valid, otherwise set to the first option
            saved_option = results_data[current_item].get("option")
            if saved_option in SIZE_OPTIONS:
                 radio_buttons.value = saved_option
            elif SIZE_OPTIONS: # If there are options, default to the first one
                radio_buttons.value = SIZE_OPTIONS[0]
            else: # No options, so no value can be set
                pass # Or handle as an error/default state

        else: # New item, reset to defaults
            color_picker.value = 'lightblue' # Default color
            if SIZE_OPTIONS:
                radio_buttons.value = SIZE_OPTIONS[0] # Default to the first radio option

        # Enable widgets in case they were disabled
        color_picker.disabled = False
        radio_buttons.disabled = False
        save_button.disabled = False
        skip_button.disabled = False
    else:
        # This case should be handled by the save button logic, but as a fallback:
        item_label.value = "All items processed! 🎉"
        definition_label.value = ""
        color_picker.disabled = True
        radio_buttons.disabled = True
        save_button.disabled = True
        skip_button.disabled = True

# --- Initial Display ---
# Connect the event handler to the button
save_button.on_click(lambda b: on_button_clicked(True))
skip_button.on_click(lambda b: on_button_clicked(False))

# Layout the widgets
widgets_layout = widgets.VBox([
    item_label,
    definition_label,
    color_picker,
    radio_buttons,
    widgets.HBox([save_button, skip_button]),
    output_area  # For feedback messages
])

# Start by displaying the first item
if ITEM_LIST:
    # Skip already annotated items
    while current_item_index < len(ITEM_LIST) and ITEM_LIST[current_item_index] in results_data:
        print(f"Skipping already annotated item: {ITEM_LIST[current_item_index]}")
        current_item_index += 1

    if current_item_index < len(ITEM_LIST):
        display_next_item()
        display(widgets_layout)
    else:
        clear_output(wait=True)
        print("All items in the list were already annotated in the JSON file.")
        print(f"Check '{OUTPUT_FILENAME}' for the results.")
        # Optionally display the widgets in a disabled state or just a message
        item_label.value = "All items previously processed! ✅"
        definition_label.value = ""
        color_picker.disabled = True
        radio_buttons.disabled = True
        save_button.disabled = True
        skip_button.disabled = True
        display(widgets_layout)

elif not ITEM_LIST:
    print("The ITEM_LIST is empty. Nothing to process.")